In [1]:
import requests
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

In [84]:
####### Helper functions

# Clean line of output from API
def clean_line(line): 
    line = line.replace('"', '')
    line = line.replace('[', '')
    line = line.replace(']', '')
    
    return line

# Clean df 
def clean_df(df):
    temp_df = df.copy()
    temp_df = temp_df[temp_df.event.isin(['Rated Blitz game', 'Rated Rapid game'])]
    temp_df = temp_df[temp_df.variant=='Standard']
    temp_df = temp_df[(temp_df.white_elo * 1.2 > temp_df.black_elo) & (temp_df.black_elo * 1.2 > temp_df.white_elo)]
    
    return temp_df

# Lag df
def get_lagged_df(df):
    df_lag_1 = df[['date', 'points']].shift(-1)
    df_lag_2 = df[['date', 'points']].shift(-2)
    df_lag_3 = df[['date', 'points']].shift(-3)

    df_lag_1.columns = ['date_lag_1', 'points_lag_1']
    df_lag_2.columns = ['date_lag_2', 'points_lag_2']
    df_lag_3.columns = ['date_lag_3', 'points_lag_3']
    
    df_concatenated = pd.concat([df, df_lag_1, df_lag_2, df_lag_3], axis=1)

    return df_concatenated


In [33]:
def get_games(player='eono619', games=100):
    
    # Make GET request
    url = "https://lichess.org/api/games/user/" + player
    
    resp = requests.get(url,
      params={
        'max': games # I'm only getting the most recent 10 puzzles you completed. You can get more if you'd like.
      },
    )
    
    # Decode and Split
    split_resp = resp.content.decode().split('\n')

    
    # Containers for relevant Information 
    game_type_container = []
    date_container = []
    white_container = []
    black_container = []
    result_container = []
    white_elo_container = []
    black_elo_container = []
    variant_container = []
    points_container = []


    # Parse through decoded response 
    for line in split_resp: 

        cleaned = clean_line(line)

        # Collect Data
        if cleaned.split(' ')[0] == 'Event':
            game_type_container.append(cleaned.split(' ', 1)[1])

        if cleaned.split(' ')[0] == 'Date':
            date_container.append(cleaned.split(' ', 1)[1])

        if cleaned.split(' ')[0] == 'White':
            white_container.append(cleaned.split(' ', 1)[1])

        if cleaned.split(' ')[0] == 'Black':
            black_container.append(cleaned.split(' ', 1)[1])

        if cleaned.split(' ')[0] == 'Result':
            result_container.append(cleaned.split(' ', 1)[1])

        if cleaned.split(' ')[0] == 'WhiteElo':
            white_elo_container.append(cleaned.split(' ', 1)[1])

        if cleaned.split(' ')[0] == 'BlackElo':
            black_elo_container.append(cleaned.split(' ', 1)[1])

        if cleaned.split(' ')[0] == 'Variant':
            variant_container.append(cleaned.split(' ', 1)[1])

    # Create DataFrame
    d = {'event': game_type_container, 
         'date': date_container, 
         'white_player': white_container, 
         'black_player': black_container, 
         'result': result_container, 
         'white_elo': white_elo_container,
         'black_elo': black_elo_container, 
         'variant': variant_container

    }
    
    df = pd.DataFrame(d)
    
    # Logic for a win
    for (result, white_player, black_player) in zip(df['result'], df['white_player'], df['black_player']):
        if result == '1-0':
            if white_player == player:
                points_container.append(1)
            else: 
                points_container.append(0)
        elif result == '0-1':
            if black_player == player:
                points_container.append(1)
            else: 
                points_container.append(0)
        else:
            points_container.append(0.5)

    df['points'] = points_container
    df = df[(df.white_elo != '?') & (df.black_elo != '?')]
    df['white_elo'] = pd.to_numeric(df['white_elo'])
    df['black_elo'] = pd.to_numeric(df['black_elo'])
    df['elo_diff'] = abs(df['white_elo'] - df['black_elo'])


    return df

In [44]:
df = get_games('eono619', 500)
df = clean_df(df)

df.head()

,event,date,white_player,black_player,result,white_elo,black_elo,variant,points,elo_diff
0,Rated Blitz game,2021.08.22,hamzaly1013,eono619,1-0,1651,1689,Standard,0.0,38
1,Rated Blitz game,2021.08.22,eono619,gnomius,1-0,1679,1640,Standard,1.0,39
2,Rated Blitz game,2021.08.22,eono619,clasherindistress,0-1,1691,1687,Standard,0.0,4
3,Rated Blitz game,2021.08.21,eono619,Isaac1969,0-1,1702,1712,Standard,0.0,10
4,Rated Blitz game,2021.08.14,dhruvasagar,eono619,1-0,1718,1713,Standard,0.0,5


In [87]:
all_df = get_lagged_df(df)
all_df.head()

,event,date,white_player,black_player,result,white_elo,black_elo,variant,points,elo_diff,date_lag_1,points_lag_1,date_lag_2,points_lag_2,date_lag_3,points_lag_3
0,Rated Blitz game,2021.08.22,hamzaly1013,eono619,1-0,1651,1689,Standard,0.0,38,2021.08.22,1.0,2021.08.22,0.0,2021.08.21,0.0
1,Rated Blitz game,2021.08.22,eono619,gnomius,1-0,1679,1640,Standard,1.0,39,2021.08.22,0.0,2021.08.21,0.0,2021.08.14,0.0
2,Rated Blitz game,2021.08.22,eono619,clasherindistress,0-1,1691,1687,Standard,0.0,4,2021.08.21,0.0,2021.08.14,0.0,2021.08.11,1.0
3,Rated Blitz game,2021.08.21,eono619,Isaac1969,0-1,1702,1712,Standard,0.0,10,2021.08.14,0.0,2021.08.11,1.0,2021.08.09,0.0
4,Rated Blitz game,2021.08.14,dhruvasagar,eono619,1-0,1718,1713,Standard,0.0,5,2021.08.11,1.0,2021.08.09,0.0,2021.08.09,1.0


In [ ]:
df.apply(lambda x: [1, 2], axis=

In [72]:
# Check whether games were played on the same day 
all_df['same_3_dates'] = all_df.apply(lambda x: x.date == x.date_lag_1 == x.date_lag_2 == x.date_lag_3, axis = 1)
all_df['same_2_dates'] = all_df.apply(lambda x: x.date == x.date_lag_1 == x.date_lag_2, axis = 1)
all_df['same_1_date'] = all_df.apply(lambda x: x.date == x.date_lag_1, axis = 1)


In [ ]:
# Check whether last 3 games where all 1, all 0 or mixed 